In [1]:
!unzip /content/eeg-data-classification.zip

Archive:  /content/eeg-data-classification.zip
  inflating: Sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [3]:
!pip -q install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.0/430.0 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import os
from sklearn.metrics import accuracy_score, classification_report

In [5]:
train_path = 'train.csv'  # Update this path if your file is in a different location
test_path = 'test.csv'    # Update this path if your file is in a different location
submission_path = 'submission.csv'  # Output path for predictions

In [6]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

print("Training data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

Training data shape: (7488, 18)
Test data shape: (7488, 18)


In [7]:
print("Missing values in training data:", train_data.isnull().sum().sum())
print("Missing values in test data:", test_data.isnull().sum().sum())

Missing values in training data: 5570
Missing values in test data: 12923


In [8]:
print("\nTraining data statistics:")
print(train_data.describe())


Training data statistics:
                id           V0           V1           V2           V3  \
count  7488.000000  7488.000000  7488.000000  6974.000000  6224.000000   
mean   3743.500000     0.502847     0.338744     0.414566     0.353979   
std    2161.743741     0.289789     0.123015     0.118579     0.113369   
min       0.000000     0.000020     0.001666     0.002030     0.000000   
25%    1871.750000     0.253722     0.271370     0.338785     0.279900   
50%    3743.500000     0.506557     0.316577     0.399992     0.345080   
75%    5615.250000     0.754679     0.371844     0.467367     0.391288   
max    7487.000000     0.999774     0.989972     1.000000     0.994595   

                V4           V5           V6           V7           V8  \
count  7488.000000  5987.000000  7488.000000  7488.000000  7488.000000   
mean      0.334531     0.233684     0.283537     0.308027     0.295842   
std       0.109179     0.108096     0.098026     0.137870     0.111823   
min       

In [9]:
X_train = train_data.drop('Class', axis=1)
y_train = train_data['Class']

In [10]:
save_path = 'agModels-eeg-eye'

In [11]:
predictor = TabularPredictor(
    label='Class',
    path=save_path,
    eval_metric='roc_auc',
    problem_type='binary'
)

In [12]:
predictor.fit(
    train_data=train_data,
    time_limit=600,
    presets='best_quality',  # Use high quality preset for better performance
    verbosity=2  # Output detailed information during training
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.39 GB / 12.67 GB (89.9%)
Disk Space Avail:   190.50 GB / 235.68 GB (80.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke

(_ray_fit pid=2209) [1000]	valid_set's binary_logloss: 0.299077
(_ray_fit pid=2345) [1000]	valid_set's binary_logloss: 0.300061 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=2483) [1000]	valid_set's binary_logloss: 0.310969 [repeated 4x across cluster]
(_ray_fit pid=2593) [1000]	valid_set's binary_logloss: 0.287974 [repeated 4x across cluster]


(_dystack pid=2078) 	0.9519	 = Validation score   (roc_auc)
(_dystack pid=2078) 	54.35s	 = Training   runtime
(_dystack pid=2078) 	2.66s	 = Validation runtime
(_dystack pid=2078) Fitting model: LightGBM_BAG_L1 ... Training model for up to 37.80s of the 86.16s of remaining time.
(_dystack pid=2078) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.08%)


(_ray_fit pid=2730) [1000]	valid_set's binary_logloss: 0.23998 [repeated 4x across cluster]
(_ray_fit pid=2834) [1000]	valid_set's binary_logloss: 0.228639 [repeated 2x across cluster]
(_ray_fit pid=2946) [1000]	valid_set's binary_logloss: 0.24499 [repeated 2x across cluster]
(_ray_fit pid=3063) [1000]	valid_set's binary_logloss: 0.238469 [repeated 2x across cluster]


(_dystack pid=2078) 	0.9661	 = Validation score   (roc_auc)
(_dystack pid=2078) 	41.25s	 = Training   runtime
(_dystack pid=2078) 	0.91s	 = Validation runtime
(_dystack pid=2078) Fitting model: WeightedEnsemble_L2 ... Training model for up to 145.05s of the 41.05s of remaining time.
(_dystack pid=2078) 	Ensemble Weights: {'LightGBM_BAG_L1': 0.929, 'LightGBMXT_BAG_L1': 0.071}
(_dystack pid=2078) 	0.9662	 = Validation score   (roc_auc)
(_dystack pid=2078) 	0.14s	 = Training   runtime
(_dystack pid=2078) 	0.0s	 = Validation runtime
(_dystack pid=2078) Fitting 108 L2 models, fit_strategy="sequential" ...
(_dystack pid=2078) Fitting model: LightGBMXT_BAG_L2 ... Training model for up to 40.89s of the 40.87s of remaining time.
(_dystack pid=2078) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.10%)
(_dystack pid=2078) 	0.9645	 = Validation score   (roc_auc)
(_dystack pid=2078) 	26.58s	 = Training   runtime
(_dysta

In [13]:
print("\nModel Leaderboard:")
leaderboard = predictor.leaderboard()
print(leaderboard)


Model Leaderboard:
                      model  score_val eval_metric  pred_time_val    fit_time  \
0       WeightedEnsemble_L3   0.974404     roc_auc       9.438795  400.178091   
1           CatBoost_BAG_L2   0.973811     roc_auc       8.516526  326.037555   
2         LightGBMXT_BAG_L2   0.973742     roc_auc       8.573931  324.544181   
3           LightGBM_BAG_L2   0.973332     roc_auc       8.497792  326.506348   
4   RandomForestEntr_BAG_L2   0.973111     roc_auc       8.656322  303.193668   
5   RandomForestGini_BAG_L2   0.972213     roc_auc       8.803559  301.103309   
6       WeightedEnsemble_L2   0.972190     roc_auc       2.020361  188.789392   
7           CatBoost_BAG_L1   0.969187     roc_auc       0.118939  135.312459   
8           LightGBM_BAG_L1   0.969106     roc_auc       1.285079   44.871536   
9   RandomForestEntr_BAG_L1   0.956620     roc_auc       0.290321    6.266278   
10        LightGBMXT_BAG_L1   0.953852     roc_auc       3.245015   56.314628   
11  Rand

In [14]:
if hasattr(predictor, 'feature_importance'):
    print("\nFeature Importance:")
    importance = predictor.feature_importance(data=train_data)
    print(importance)

Computing feature importance via permutation shuffling for 17 features using 5000 rows with 5 shuffle sets...



Feature Importance:


/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you t

KeyboardInterrupt: 

In [15]:
y_pred = predictor.predict(test_data)
print("\nPrediction distribution:")
print(y_pred.value_counts())

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you t


Prediction distribution:
Class
0    4252
1    3236
Name: count, dtype: int64


In [16]:
submission = pd.DataFrame()
submission['id'] = range(len(test_data))
submission['Class'] = y_pred
submission.to_csv(submission_path, index=False)
print(f"\nSubmission file saved to {submission_path}")


Submission file saved to submission.csv
